# Worksheet 08

Name: Akshat Gurbuxani


UID: U80105862

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [2]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[4.6534465354729955, 4.8818589975605295, 4.537416778830336, 4.990697858860169, 5.002638881457441, 5.531687026609374, 7.292602053313279, 4.995330735815908, 5.48284646011562, 4.242716721459701]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
data_points = 10
mean = 8
stdev = 1
s2 = np.random.normal(mean, stdev, data_points).tolist()
print(s2)

[8.482797578073573, 7.765988347248856, 7.664981075738338, 8.581067604655383, 8.184483547251544, 6.915616681973214, 8.343364020300621, 7.01932041795256, 10.350124943921404, 6.807074703405841]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[6.807074703405841, 10.350124943921404, 4.242716721459701, 7.01932041795256, 8.343364020300621, 5.48284646011562, 6.915616681973214, 4.995330735815908, 7.292602053313279, 8.184483547251544]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

1. Prior probability, which is, here, the probability of choosing one of the two distributions (0.5).
2. Mean of the distribution.
3. Variance of the distribution.

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2)]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

/Users/akshatgurbuxani/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


[6.807074703405841, 10.350124943921404, 7.01932041795256, 8.343364020300621, 6.915616681973214, 7.292602053313279, 8.184483547251544]
[4.242716721459701, 5.48284646011562, 4.995330735815908]
P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 7.844655195445496,  mean_2 = 4.90696463913041
var_1 = 1.3667426378372765,  var_2 = 0.26022457830484846


They are pretty close to the true values.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

# Lists to store probabilities
prob_s0_x = [] 
prob_s1_x = [] 

k = 2  # Number of clusters

for p in data:
    print("Point = ", p)
    pdf_i = []

    for j in range(k):
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print(f"Probability of observing that point if it came from cluster {j} = {pdf_i[j]}")

    prob_x = sum(prob_s[j] * pdf_i[j] for j in range(k))

    prob_s0_x.append(pdf_i[0] * prob_s[0] / prob_x)
    prob_s1_x.append(pdf_i[1] * prob_s[1] / prob_x)

probs = zip(data, prob_s0_x, prob_s1_x)

for p in probs:
    print(f"Data Point: {p[0]}")
    print(f"Probability of coming from S_1: {p[1]}")
    print(f"Probability of coming from S_2: {p[2]}\n")

Point =  6.807074703405841
Probability of observing that point if it came from cluster 0 = 0.218814035273288
Probability of observing that point if it came from cluster 1 = 4.055558510572887e-12
Point =  10.350124943921404
Probability of observing that point if it came from cluster 0 = 0.05438741767898043
Probability of observing that point if it came from cluster 1 = 1.50526765486769e-95
Point =  4.242716721459701
Probability of observing that point if it came from cluster 0 = 0.009058212726988372
Probability of observing that point if it came from cluster 1 = 0.058977544934515784
Point =  7.01932041795256
Probability of observing that point if it came from cluster 0 = 0.2432420477864768
Probability of observing that point if it came from cluster 1 = 7.535932969264615e-15
Point =  8.343364020300621
Probability of observing that point if it came from cluster 0 = 0.2730936082646331
Probability of observing that point if it came from cluster 1 = 2.080694524111701e-38
Point =  5.482846460

Probabiliy of coming from S_1 is higher than S_2. As expected, close to the truth.

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [sum([x[0] * ((x[1] - mean[0]) ** 2) for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * ((x[1] - mean[1]) ** 2) for x in zip(prob_s1_x, data)]) / sum(prob_s1_x)]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.7,  P(S_2) = 0.3
mean_1 = 7.543903317379604,  mean_2 = 4.84279211142856
var_1 = 1.9976193230087063,  var_2 = 0.22205030226242742


The result of the code demonstrates a significant change in the classification likelihood and the variable value when compared to k-means. Specifically, the likelihood of classification into S_2 has notably increased, while the variable is considerably lower in this new context. This suggests a substantial shift in the clustering results when using the modified approach presented in the code.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [8]:
prob_c0_x = []
prob_c1_x = []
prob_x = []

k = 2

for p in data:
    pdf_i = [norm.pdf(p, mean[j], np.sqrt(var[j])) for j in range(k)]
    prob_x = sum(prob_c[j] * pdf_i[j] for j in range(k))

    prob_c0_x.append(pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append(pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)

for p in probs:
    print(f"Data Point: {p[0]}")
    print(f"Probability Class 1: {p[1]}")
    print(f"Probability Class 2: {p[2]}\n")


Data Point: 6.807074703405841
Probability Class 1: 0.9998414511329046
Probability Class 2: 0.00015854886709536174

Data Point: 10.350124943921404
Probability Class 1: 1.0
Probability Class 2: 1.286651720912174e-29

Data Point: 4.242716721459701
Probability Class 1: 0.15189312808685537
Probability Class 2: 0.8481068719131447

Data Point: 7.01932041795256
Probability Class 1: 0.9999795045736538
Probability Class 2: 2.0495426346193917e-05

Data Point: 8.343364020300621
Probability Class 1: 0.9999999999989989
Probability Class 2: 1.001096478685369e-12

Data Point: 5.48284646011562
Probability Class 1: 0.5140614160470725
Probability Class 2: 0.48593858395292755

Data Point: 6.915616681973214
Probability Class 1: 0.9999430386037848
Probability Class 2: 5.696139621524411e-05

Data Point: 4.995330735815908
Probability Class 1: 0.2016035162229683
Probability Class 2: 0.7983964837770317

Data Point: 7.292602053313279
Probability Class 1: 0.9999988721796276
Probability Class 2: 1.1278203724160376

Class 1 certainity is higher than class 2

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [9]:
probs = zip(data, prob_c0_x, prob_c1_x)

for p in probs:
    print(p[0])

    if p[1] > p[2]:
        print("Cluster #1\n")
    elif p[1] < p[2]:
        print("Cluster #2\n")
    else:
        print("Ambiguous\n")
    


6.807074703405841
Cluster #1

10.350124943921404
Cluster #1

4.242716721459701
Cluster #2

7.01932041795256
Cluster #1

8.343364020300621
Cluster #1

5.48284646011562
Cluster #1

6.915616681973214
Cluster #1

4.995330735815908
Cluster #2

7.292602053313279
Cluster #1

8.184483547251544
Cluster #1

